# Atomata results analysis

## Read from database

In [1]:
import pandas as pd
import sqlite3

from python.repository import state_vectors_by_run_id, total_runs

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("results.db3")

state_vectors = state_vectors_by_run_id(3, con)
total_runs = total_runs(con)

con.close()

KeyError: 0

In [7]:
total_runs

,MAX(run_id)
0,21
